In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import string
import re
from num2words import num2words

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=16)

from functools import lru_cache

import pymorphy3
import random

morph = pymorphy3.MorphAnalyzer()
punctuation = string.punctuation

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
markup = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}

In [4]:
with open('multinerd_train_ru.tsv') as f:
    lines = f.readlines()
    df_train = pd.DataFrame(lines)

In [5]:
with open('multinerd_test_ru.tsv') as f:
    lines = f.readlines()
    df_test = pd.DataFrame(lines)

In [6]:
with open('multinerd_dev_ru.tsv') as f:
    lines = f.readlines()
    df_val= pd.DataFrame(lines)

In [7]:
df = pd.concat([df_train, df_test, df_val], ignore_index=True)

In [8]:
df['split'] = df[0].apply(lambda x: x.split('\t'))

In [9]:
df['word_index'] = df['split'].apply(lambda x: x[0])
df['word'] = [row[1] if len(row) > 1 else '' for row in df['split']]

In [10]:
df['token'] = [row[2] if len(row) > 2 else '' for row in df['split']]
df['token'] = df['token'].apply(lambda x: x.replace('\n', ''))

In [11]:
df.head()

,0,split,word_index,word,token
0,0\tСуществовали\tO\n,"[0, Существовали, O\n]",0,Существовали,O
1,1\tвиды\tO\n,"[1, виды, O\n]",1,виды,O
2,"2\t,\tO\n","[2, ,, O\n]",2,",",O
3,3\tнапоминавшие\tO\n,"[3, напоминавшие, O\n]",3,напоминавшие,O
4,4\tвследствие\tO\n,"[4, вследствие, O\n]",4,вследствие,O


In [12]:
df['token_BI'] = [row.split('-')[0] if len(row.split('-')) > 1 else '' for row in df['token']]

In [13]:
df['token_NER'] = [row.split('-')[1] if len(row.split('-')) > 1 else '' for row in df['token']]

In [14]:
df['token_BI'].value_counts()

     1293090
B     115216
I      77441
Name: token_BI, dtype: int64

In [15]:
df['token_NER'].value_counts()

         1293090
PER        69761
LOC        51110
ORG        17613
EVE        16339
ANIM       10017
MEDIA       8305
PLANT       5223
DIS         4010
TIME        3954
FOOD        2343
CEL         2071
INST         671
VEHI         618
MYTH         594
BIO           28
Name: token_NER, dtype: int64

In [16]:
split_keys = [key.split('-')[1] if len(key.split('-'))>1 else '' for key in markup.keys()]
df['token_fixed'] = ['MISC' if token not in split_keys else token for token in df['token_NER']]

In [17]:
df['token_fixed'].value_counts()

        1293090
PER       69761
MISC      54173
LOC       51110
ORG       17613
Name: token_fixed, dtype: int64

In [18]:
df.drop('token_NER', inplace=True, axis=1)

In [19]:
df['ner_token'] = df['token_BI'] + '-' + df['token_fixed']

In [20]:
df.drop('token_BI', inplace=True, axis=1)
df.drop('token_fixed', inplace=True, axis=1)
df['ner_token'][df['ner_token'] == '-'] = 'O'

In [21]:
df.drop(0, inplace=True, axis=1)
df.drop('split', inplace=True, axis=1)

In [22]:
df.drop('token', inplace=True, axis=1)

In [23]:
df.ner_token.value_counts()

O         1293090
B-LOC       41293
I-PER       35068
B-PER       34693
B-MISC      31195
I-MISC      22978
I-LOC        9817
I-ORG        9578
B-ORG        8035
Name: ner_token, dtype: int64

In [24]:
df

,word_index,word,ner_token
0,0,Существовали,O
1,1,виды,O
2,2,",",O
3,3,напоминавшие,O
4,4,вследствие,O
...,...,...,...
1485742,34,за,O
1485743,35,право,O
1485744,36,спариваться,O
1485745,37,.,O


In [25]:
df['ner_token'] = [markup[token] for token in df['ner_token']]

In [26]:
def create_main(df):
    main = {'tokens': [], 'ner_tags': []}
    length = len(df)
    tokens = []
    ner_tokens = []
    for i in range(length):
        index = df['word_index'][i]
        if index != '\n':
            tokens.append(df['word'][i])
            ner_tokens.append(df['ner_token'][i])
                
        elif index == '\n':
            main['tokens'].append(tokens)
            main['ner_tags'].append(ner_tokens)
            tokens, ner_tokens = [], []
                
    return main

In [27]:
df_main = create_main(df)
df_main = pd.DataFrame(df_main)

In [28]:
# Function to check if number of tokens equals number of ner tags for a given row
def length_check(tokens, ner_tags):
    return 'ok' if len(tokens) == len(ner_tags) else 'WRONG'

In [29]:
df_main['len_check'] = df_main.apply(lambda row: length_check(row['tokens'], row['ner_tags']), axis=1)

In [30]:
df_main['len_check'].value_counts()

ok    82858
Name: len_check, dtype: int64

In [31]:
df_main['augmented_tokens'] = df_main['tokens']
df_main['augmented_ner_tags'] = df_main['ner_tags']

In [32]:
# Takes a list of tokens, and changes 1 or 2 (based on number of tokens)
# random tokens to its lemma, simulating transcribed speech system mistake,
# with the following conditions:
# -  original token should be at elast 7 characters long,
#    since transcribed speech system will probably recognize short words more accurately
# -  token should not start with uppercase, to aviod changing unique names
# -  the original token and its lemma must not differ in more than 5 characters.
#    since transcribed words should not be too different (assuming we arent using utter trash
#    transcription system.)
def simulate_mistakes(tokens):
    suitable_for_augmentation = []
    
    @lru_cache(maxsize=512)
    def get_lemma(word):
        parsed_word = morph.parse(word)[0]
        lemma = parsed_word.normal_form
        return lemma
    
    def lemmatize_tokens(tokens, suitable_for_augmentation, k):
        to_augment = random.sample(suitable_for_augmentation, k)
        for i in to_augment:
            tokens[i] = get_lemma(tokens[i])
    
    for i in range(len(tokens)):
        word = tokens[i]
        if len(word) > 6 and not word[0].isupper():
            lemma = get_lemma(word)
            if lemma != word:
                difference_counter = abs(len(word) - len(lemma))
                for index in range(min(len(word), len(lemma))):
                    if word[index] != lemma[index]:
                        difference_counter += 1

                if difference_counter < 6:
                        suitable_for_augmentation.append(i)
                    
    if suitable_for_augmentation:
        if len(tokens) <=10:
            lemmatize_tokens(tokens, suitable_for_augmentation, 1)
        elif len(suitable_for_augmentation) > 3:
            lemmatize_tokens(tokens, suitable_for_augmentation, 2)
       
    return tokens

In [33]:
df_main['augmented_tokens'] = df_main['augmented_tokens'].parallel_apply(lambda x: simulate_mistakes(x))

In [34]:
# Function that splits tokens by hyphen (since hyphen is not going to be in trascribed speech)
# and adjusts the ner_tags accordingly (iserting same tags, as the original token had)
def split_hyphen(tokens, ner_tags):
    
    for i in range(len(tokens)):
        if '-' in tokens[i] or '—' in tokens[i]:
            new_tokens = tokens[i].split('-')
            new_tokens = tokens[i].split('—')
            tag_to_insert = ner_tags[i]
            del ner_tags[i]
            del tokens[i] 
            index = i
            
            for _ in range(len(new_tokens)):
                ner_tags.insert(index, tag_to_insert)
                tokens.insert(index, new_tokens[_])
                index += 1 
               
    return tokens, ner_tags

In [35]:
df_main[['augmented_tokens', 'augmented_ner_tags']] = df_main.parallel_apply(lambda row: pd.Series(split_hyphen(row['tokens'], row['ner_tags'])), axis=1)

In [36]:
df_main['augmented_length_check'] = df_main.apply(lambda row: length_check(row['augmented_tokens'], row['augmented_ner_tags']), axis=1)
df_main['augmented_length_check'].value_counts()

ok    82858
Name: augmented_length_check, dtype: int64

In [37]:
df_main.head()

,tokens,ner_tags,len_check,augmented_tokens,augmented_ner_tags,augmented_length_check
0,"[Существовали, виды, ,, напоминавшие, вследств...","[0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 7, 0, 0, 0, 0, ...",ok,"[Существовали, виды, ,, напоминавшие, вследств...","[0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 7, 0, 0, 0, 0, ...",ok
1,"[Экранизация, одноимённого, произведения, 1982...","[0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0]",ok,"[Экранизация, одноимённого, произведения, 1982...","[0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0]",ok
2,"["", Карликовые, гремучники, "", (, ), -, род, я...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 7, 0, 0, 0]",ok,"["", Карликовые, гремучники, "", (, ), -, род, я...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 7, 0, 0, 0]",ok
3,"[В, 1638, году, на, берегах, реки, Делавэр, бы...","[0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 5, 6, 0, 0, 5, ...",ok,"[В, 1638, году, на, берегах, реки, Делавэр, бы...","[0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 5, 6, 0, 0, 5, ...",ok
4,"[Кроме, этого, ,, распространён, на, западном,...","[0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 0, 0, 0, 0, 0, 0]",ok,"[Кроме, этого, ,, распространён, на, западном,...","[0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 0, 0, 0, 0, 0, 0]",ok


In [38]:
def cubes_squares_punct_etc(tokens, ner_tags):
    
    def switch_ner_tags(ner_tags, i):
        ner_tags[i-1], ner_tags[i] = ner_tags[i], ner_tags[i-1]
        
    def switch_tokens(tokens, i):
        tokens[i-1], tokens[i] = tokens[i], tokens[i-1]
        
    def m_km(tokens, i):
        if tokens[i-1] == 'километр':
            switch_tokens(tokens, i)
            switch_ner_tags(ner_tags, i)
        elif tokens[i-1] == 'метр':
            switch_tokens(tokens, i)
            switch_ner_tags(ner_tags, i)
    
    square, cube, half, third = '²', '³', '½', '⅓'
    thousand, million, billion = 'тыс', 'млн', 'млрд'
    km, m = 'км', 'м'
   
    for i in range(len(tokens)):
        tokens[i] = re.sub(r'[^\w\s]+', '', tokens[i])
        if tokens[i] == half: tokens[i] = 'с половиной'
        elif tokens[i] == third: tokens[i] = 'одна треть'
        elif tokens[i] == thousand: tokens[i] = 'тысяч'
        elif tokens[i] == million: tokens[i] = 'миллионов'
        elif tokens[i] == billion: tokens[i] = 'миллиадров'
        elif tokens[i] == km: tokens[i] = 'километр'
        elif tokens[i] == m: tokens[i] = 'метр'
        
        elif tokens[i] == square:
            tokens[i] = 'квадратный'
            m_km(tokens, i)
        
        elif tokens[i] == cube:
            tokens[i] = 'кубический'
            m_km(tokens, i)
            
    return tokens, ner_tags

In [39]:
df_main[['augmented_tokens', 'augmented_ner_tags']] = df_main.parallel_apply(lambda row: pd.Series(cubes_squares_punct_etc(row['augmented_tokens'], row['augmented_ner_tags'])), axis=1)

In [40]:
df_main['augmented_length_check'] = df_main.apply(lambda row: length_check(row['augmented_tokens'], row['augmented_ner_tags']), axis=1)
df_main['augmented_length_check'].value_counts()

ok    82858
Name: augmented_length_check, dtype: int64

In [41]:
df_main.head()

,tokens,ner_tags,len_check,augmented_tokens,augmented_ner_tags,augmented_length_check
0,"[Существовали, виды, ,, напоминавшие, вследств...","[0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 7, 0, 0, 0, 0, ...",ok,"[Существовали, виды, , напоминавшие, вследстви...","[0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 7, 0, 0, 0, 0, ...",ok
1,"[Экранизация, одноимённого, произведения, 1982...","[0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0]",ok,"[Экранизация, одноимённого, произведения, 1982...","[0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0]",ok
2,"["", Карликовые, гремучники, "", (, ), -, род, я...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 7, 0, 0, 0]",ok,"[, Карликовые, гремучники, , , , , род, ядовит...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 7, 0, 0, 0]",ok
3,"[В, 1638, году, на, берегах, реки, Делавэр, бы...","[0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 5, 6, 0, 0, 5, ...",ok,"[В, 1638, году, на, берегах, реки, Делавэр, бы...","[0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 5, 6, 0, 0, 5, ...",ok
4,"[Кроме, этого, ,, распространён, на, западном,...","[0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 0, 0, 0, 0, 0, 0]",ok,"[Кроме, этого, , распространён, на, западном, ...","[0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 0, 0, 0, 0, 0, 0]",ok


In [42]:
def filter_non_words(tokens, ner_tags):
    
    # Remove 'г's, using that just in case there are multiple occurences in one row
    if 'г' in tokens:
        g_indexes = [index for index, value in enumerate(tokens) if value == 'г']
        tokens = [value for index, value in enumerate(tokens) if index not in g_indexes]
        
    filtered_tags = [tag if token.isalnum() else np.nan for token, tag in zip(tokens, ner_tags)]
    filtered_tokens = [token for token in tokens if token != '' ]
    return filtered_tokens, [tag for tag in filtered_tags if pd.notnull(tag)]

In [43]:
df_main[['augmented_tokens', 'augmented_ner_tags']] = df_main.parallel_apply(lambda row: pd.Series(filter_non_words(row['augmented_tokens'], row['augmented_ner_tags'])), axis=1)

In [44]:
df_main['augmented_length_check'] = df_main.apply(lambda row: length_check(row['augmented_tokens'], row['augmented_ner_tags']), axis=1)
df_main['augmented_length_check'].value_counts()

ok       82836
WRONG       22
Name: augmented_length_check, dtype: int64

In [45]:
df_main[df_main['augmented_length_check']== 'WRONG']

,tokens,ner_tags,len_check,augmented_tokens,augmented_ner_tags,augmented_length_check
1218,"[Ещё, одним, источником, романа, послужила, но...","[0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0, ...",ok,"[Ещё, одним, источником, романа, послужила, но...","[0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0]",WRONG
7074,"[для, «, Библиотеки, всемирной, литературы, »,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, ...",ok,"[для, Библиотеки, всемирной, литературы, подго...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0]",WRONG
9271,"[Порядок, судебного, поединка, в, Чехии, регла...","[0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",ok,"[Порядок, судебного, поединка, в, Чехии, регла...","[0, 0, 0, 0, 5, 0, 0, 0, 0]",WRONG
12708,"[На, 2, очка, отстали, Смейкал, и, Андерссон, ...","[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0]",ok,"[На, 2, очка, отстали, Смейкал, и, Андерссон, ...","[0, 0, 0, 0, 1, 0, 1, 0, 0, 1]",WRONG
13183,"[Со, счётом, 3, ½, :, 6, ½, уступил, звание, ч...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",ok,"[Со, счётом, 3, с половиной, 6, с половиной, у...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5]",WRONG
13188,"[В, ноябре, 2014, года, в, российском, городе,...","[0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, ...",ok,"[В, ноябре, 2014, года, в, российском, городе,...","[0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, ...",WRONG
22591,"[Он, выиграл, три, партии, подряд, с, 11-го, п...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, ...",ok,"[Он, выиграл, три, партии, подряд, с, 11го, по...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, ...",WRONG
23379,"[Автор, методического, труда, о, скрипичной, и...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",ok,"[Автор, методического, труда, о, скрипичной, и...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, ...",WRONG
24486,"[Древнеримский, учёный, Плиний, Старший, (, "",...","[0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",ok,"[Древнеримский, учёный, Плиний, Старший, Естес...","[0, 0, 1, 2, 0, 0, 0]",WRONG
36630,"[Это, музыкальная, обработка, сценария, Федери...","[0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",ok,"[Это, музыкальная, обработка, сценария, Федери...","[0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0]",WRONG


In [46]:
df_main.drop(df_main[df_main['augmented_length_check'] == 'WRONG'].index, inplace=True)
df_main.reset_index(drop=True, inplace=True)

In [47]:
def look_for_trash(trash):
    for i in range(len(df_main)):
        if trash in df_main['augmented_tokens'][i]:
            print(i)
            print(df_main['augmented_tokens'][i])

In [48]:
trash_1 = '₆'
look_for_trash(trash_1)

3309
['Селективные', 'антагонисты', '5HT', '₆', 'рецепторов', 'такие', 'как', 'латрепирдин', 'идалопирдин', 'Lu', 'AE58054', 'и', 'SB742', '457', 'в', 'настоящее', 'время', 'проходят', 'клинические', 'испытания', 'в', 'качестве', 'потенциальных', 'новых', 'средств', 'лечения', 'болезни', 'Альцгеймера', 'и', 'других', 'форм', 'деменции']
74469
['В', 'последнее', 'время', 'селективные', 'агонисты', '5HT', '₆', 'рецепторов', 'WAY181', '187', 'и', 'WAY208', '466', 'показали', 'активность', 'в', 'моделях', 'на', 'крысах', 'таких', 'психических', 'заболеваний', 'как', 'большая', 'депрессия', 'тревожные', 'расстройства', 'обсессивнокомпульсивное', 'расстройство', 'ОКР']


In [49]:
del df_main['augmented_tokens'][3309][3]
del df_main['augmented_ner_tags'][3309][3]

In [50]:
del df_main['augmented_tokens'][74469][6]
del df_main['augmented_ner_tags'][74469][6]

In [51]:
trash_2 = '₁'
look_for_trash(trash_2)

11644
['И', 'напротив', 'было', 'показано', 'что', 'антагонисты', '5HT', '₁', 'рецепторов', 'такие', 'как', 'лекозотан', 'способны', 'улучшать', 'определённые', 'аспекты', 'обучаемости', 'и', 'памяти', 'по', 'крайней', 'мере', 'в', 'опытах', 'на', 'крысах', 'и', 'как', 'следствие', 'в', 'настоящее', 'время', 'изучаются', 'в', 'качестве', 'потенциальных', 'кандидатов', 'на', 'лекарства', 'для', 'лечения', 'болезни', 'Альцгеймера']


In [52]:
del df_main['augmented_tokens'][11644][7]
del df_main['augmented_ner_tags'][11644][7]

In [53]:
trash_3 = '⅔'
look_for_trash(trash_3)

29534
['Из', '6', 'территории', 'страны', 'покрытых', 'лесом', 'около', '⅔', 'являются', 'искусственными', 'насаждениями', 'а', 'также', 'тысячи', 'видов', 'насекомых', 'включая', 'более', 'ста', 'видов', 'бабочек']


In [54]:
df_main['augmented_tokens'][29534].index(trash_3)

7

In [55]:
df_main['augmented_tokens'][29534].insert(7, 'двух')
df_main['augmented_tokens'][29534].insert(8, 'третей')

In [56]:
del df_main['augmented_tokens'][29534][9]

In [57]:
tag_to_indert = df_main['augmented_ner_tags'][29534][7]

In [58]:
df_main['augmented_ner_tags'][29534].insert(7, tag_to_indert)

In [59]:
len(df_main)

82836

In [60]:
def replace_numerical_with_words(tokens, ner_tags):
    adjusted_ner_tags = []
    
    for i, token in enumerate(tokens):
        if token.isnumeric():
            index = i
            new_tokens = num2words(int(token), lang='ru').split()
            tag_to_insert = ner_tags[i]
            count = len(new_tokens)
            del ner_tags[i]
            del tokens[i] 
            for _ in range(len(new_tokens)):
                ner_tags.insert(index, tag_to_insert)
                tokens.insert(index, new_tokens[_])
                index += 1 
                
    return tokens, ner_tags

In [61]:
df_main[['augmented_tokens', 'augmented_ner_tags']] = df_main.parallel_apply(lambda row: pd.Series(replace_numerical_with_words(row['augmented_tokens'], row['augmented_ner_tags'])), axis=1)

In [62]:
df_main

,tokens,ner_tags,len_check,augmented_tokens,augmented_ner_tags,augmented_length_check
0,"[Существовали, виды, ,, напоминавшие, вследств...","[0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 7, 0, 0, 0, 0, ...",ok,"[Существовали, виды, напоминавшие, вследствие,...","[0, 0, 0, 0, 0, 0, 0, 7, 7, 0, 0, 0, 0, 0, 0]",ok
1,"[Экранизация, одноимённого, произведения, 1982...","[0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0]",ok,"[Экранизация, одноимённого, произведения, одна...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2]",ok
2,"["", Карликовые, гремучники, "", (, ), -, род, я...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 7, 0, 0, 0]",ok,"[Карликовые, гремучники, род, ядовитых, змей, ...","[0, 0, 0, 0, 7, 0, 7, 0, 0]",ok
3,"[В, 1638, году, на, берегах, реки, Делавэр, бы...","[0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 5, 6, 0, 0, 5, ...",ok,"[В, одна, тысяча, шестьсот, тридцать, восемь, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 5, ...",ok
4,"[Кроме, этого, ,, распространён, на, западном,...","[0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 0, 0, 0, 0, 0, 0]",ok,"[Кроме, этого, распространён, на, западном, бе...","[0, 0, 0, 0, 0, 0, 5, 6, 0, 0, 0, 0, 0]",ok
...,...,...,...,...,...,...
82831,"[Дебютировал, в, матче, 1, тура, против, «, Ми...","[0, 0, 0, 0, 0, 0, 3, 4, 4, 0]",ok,"[Дебютировал, в, матче, один, тура, против, Ми...","[0, 0, 0, 0, 0, 0, 4]",ok
82832,"[Дебют, состоялся, 26, туре, против, «, Сибири...","[0, 0, 0, 0, 0, 3, 4, 4, 0]",ok,"[Дебют, состоялся, двадцать, шесть, туре, прот...","[0, 0, 0, 0, 0, 0, 4]",ok
82833,"[Эктор, также, представлял, свою, страну, на, ...","[0, 0, 0, 0, 0, 0, 7, 8, 8, 0, 7, 8, 8, 8, 0]",ok,"[Эктор, также, представлял, свою, страну, на, ...","[0, 0, 0, 0, 0, 0, 7, 8, 8, 8, 8, 0, 7, 8, 8, ...",ok
82834,"[Q, дал, альбому, четыре, звезды, из, пяти, .]","[7, 0, 0, 0, 0, 0, 0, 0]",ok,"[Q, дал, альбому, четыре, звезды, из, пяти]","[7, 0, 0, 0, 0, 0, 0]",ok


In [63]:
df_main['augmented_length_check'] = df_main.apply(lambda row: length_check(row['augmented_tokens'], row['augmented_ner_tags']), axis=1)
df_main['augmented_length_check'].value_counts()

ok    82836
Name: augmented_length_check, dtype: int64

In [67]:
def lowercasing(tokens):
    return [token.lower() for token in tokens]

In [68]:
df_main['augmented_tokens'] = df_main['augmented_tokens'].parallel_apply(lambda x: lowercasing(x))

In [69]:
df_main.to_csv('MultiNERD_augmented.csv', index=False)

In [70]:
df = pd.read_csv('MultiNERD_augmented.csv')

In [71]:
df

,tokens,ner_tags,len_check,augmented_tokens,augmented_ner_tags,augmented_length_check
0,"['Существовали', 'виды', ',', 'напоминавшие', ...","[0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 7, 0, 0, 0, 0, ...",ok,"['существовали', 'виды', 'напоминавшие', 'всле...","[0, 0, 0, 0, 0, 0, 0, 7, 7, 0, 0, 0, 0, 0, 0]",ok
1,"['Экранизация', 'одноимённого', 'произведения'...","[0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0]",ok,"['экранизация', 'одноимённого', 'произведения'...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2]",ok
2,"['""', 'Карликовые', 'гремучники', '""', '(', ')...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 7, 0, 0, 0]",ok,"['карликовые', 'гремучники', 'род', 'ядовитых'...","[0, 0, 0, 0, 7, 0, 7, 0, 0]",ok
3,"['В', '1638', 'году', 'на', 'берегах', 'реки',...","[0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 5, 6, 0, 0, 5, ...",ok,"['в', 'одна', 'тысяча', 'шестьсот', 'тридцать'...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 5, ...",ok
4,"['Кроме', 'этого', ',', 'распространён', 'на',...","[0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 0, 0, 0, 0, 0, 0]",ok,"['кроме', 'этого', 'распространён', 'на', 'зап...","[0, 0, 0, 0, 0, 0, 5, 6, 0, 0, 0, 0, 0]",ok
...,...,...,...,...,...,...
82831,"['Дебютировал', 'в', 'матче', '1', 'тура', 'пр...","[0, 0, 0, 0, 0, 0, 3, 4, 4, 0]",ok,"['дебютировал', 'в', 'матче', 'один', 'тура', ...","[0, 0, 0, 0, 0, 0, 4]",ok
82832,"['Дебют', 'состоялся', '26', 'туре', 'против',...","[0, 0, 0, 0, 0, 3, 4, 4, 0]",ok,"['дебют', 'состоялся', 'двадцать', 'шесть', 'т...","[0, 0, 0, 0, 0, 0, 4]",ok
82833,"['Эктор', 'также', 'представлял', 'свою', 'стр...","[0, 0, 0, 0, 0, 0, 7, 8, 8, 0, 7, 8, 8, 8, 0]",ok,"['эктор', 'также', 'представлял', 'свою', 'стр...","[0, 0, 0, 0, 0, 0, 7, 8, 8, 8, 8, 0, 7, 8, 8, ...",ok
82834,"['Q', 'дал', 'альбому', 'четыре', 'звезды', 'и...","[7, 0, 0, 0, 0, 0, 0, 0]",ok,"['q', 'дал', 'альбому', 'четыре', 'звезды', 'и...","[7, 0, 0, 0, 0, 0, 0]",ok
